In [1]:
# IMPORTS

import numpy as np
import pandas as pd




In [2]:
# Ouverture

df_train = pd.read_csv('Corona_NLP_train.csv', sep=',', encoding='latin')

df_test = pd.read_csv('Corona_NLP_test.csv', sep=',', encoding='latin')



In [42]:
df_train.shape

(41157, 5)

In [3]:
df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
# PREPROCESSING

On vérifie qu'il n'y a pas de null (autre que location) et de ligne dupliquées

In [5]:
df_train.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [6]:
df_train = df_train.drop(['Location'], axis=1)
df_test = df_test.drop(['Location'], axis=1)



In [7]:
df_train['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [8]:
df_train.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [9]:
train_df2 = df_train.dropna()
test_df2 = df_test.dropna()



In [10]:
train_df2.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [11]:
train_df3 = train_df2.drop_duplicates()
test_df3 = test_df2.drop_duplicates()

In [12]:
train_df3.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys


In [13]:
import preprocessor as p


In [14]:
def preprocess_tweet(row):
    text = row['OriginalTweet']
    text = p.clean(text)
    return text

In [15]:
train_df3['OriginalTweet'] = train_df3.apply(preprocess_tweet, axis=1)
test_df3['OriginalTweet'] = test_df3.apply(preprocess_tweet, axis=1)

In [16]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,and and,Neutral
1,3800,48752,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,"Me, ready to go at supermarket during the outb...",Extremely Negative


In [17]:
train_df3['OriginalTweet'] = train_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
test_df3['OriginalTweet'] = test_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

In [18]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,and and,Neutral
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,me ready to go at supermarket during the outbr...,Extremely Negative


In [19]:
def give_number_to_class(row):
    sent = row['Sentiment']

    if sent=='Extremely Negative':
        return 0
    elif sent=='Negative':
        return 1
    elif sent=='Neutral':
        return 2
    elif sent=='Positive':
        return 3
    else:
        return 4


    

In [20]:
train_df3['Sentiment_Number'] = train_df3.apply(give_number_to_class, axis=1)
test_df3['Sentiment_Number'] = test_df3.apply(give_number_to_class, axis=1)



In [21]:
train_df3.head()

,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment,Sentiment_Number
0,3799,48751,16-03-2020,and and,Neutral,2
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive,3
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive,3
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive,3
4,3803,48755,16-03-2020,me ready to go at supermarket during the outbr...,Extremely Negative,0


In [22]:
np.unique(train_df3["Sentiment"].values)

array(['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral',
       'Positive'], dtype=object)

In [23]:
X_train = train_df3['OriginalTweet']  
X_test = test_df3['OriginalTweet']  

y_train = train_df3['Sentiment_Number']
y_test = test_df3['Sentiment_Number']

In [24]:
X_train[0]

'and and'

In [25]:
df_train.isnull().sum()

UserName         0
ScreenName       0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [26]:
df_train['TweetAt'].values

array(['16-03-2020', '16-03-2020', '16-03-2020', ..., '14-04-2020',
       '14-04-2020', '14-04-2020'], dtype=object)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(41157, 35525)

In [28]:
X_train_tfidf[1]


<1x35525 sparse matrix of type '<class 'numpy.float64'>'
	with 33 stored elements in Compressed Sparse Row format>

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from sklearn.linear_model import SGDClassifier


# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', SGDClassifier())])

In [45]:
predictions = text_clf.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score, classification_report

train_acc = round(accuracy_score(predictions, y_test)*100)
train_acc

54

In [40]:
len(y_test)

3798

In [47]:
print(classification_report(predictions, y_test, target_names=train_df3['Sentiment'].unique()))

                    precision    recall  f1-score   support

           Neutral       0.76      0.53      0.62       850
          Positive       0.32      0.55      0.40       609
Extremely Negative       0.79      0.53      0.64       924
          Negative       0.32      0.54      0.40       552
Extremely Positive       0.78      0.54      0.64       863

          accuracy                           0.54      3798
         macro avg       0.59      0.54      0.54      3798
      weighted avg       0.64      0.54      0.56      3798



In [ ]:
X_test[0]

'trending new yorkers encounter empty supermarket shelves pictured wegmans in brooklyn sold out online grocers foodkick maxdelivery as fearing shoppers stock up'

In [ ]:
# ACCURACY
c=0
num=len(predictions)
for k in range(num):
    if(predictions[k]==y_test[k]):
        c+=1

print(f"pourcentage de reussite : {(c*100)/num} ")




pourcentage de reussite : 57.24065297525013 


In [ ]:
# L2 ERROR

num=len(predictions)
err_tot = 0
for k in range(num):
    #print(predictions[k])
    if(k<20):
        print(f"prediction {predictions[k]}  , realité : {y_test[k]}")
    err_tot += np.square(predictions[k]-y_test[k])
err_tot/=num
err_tot = np.sqrt(err_tot) 
err_tot

prediction 2  , realité : 0
prediction 3  , realité : 3
prediction 4  , realité : 4
prediction 1  , realité : 1
prediction 2  , realité : 2
prediction 2  , realité : 2
prediction 4  , realité : 3
prediction 1  , realité : 2
prediction 0  , realité : 0
prediction 3  , realité : 4
prediction 4  , realité : 3
prediction 1  , realité : 0
prediction 0  , realité : 0
prediction 4  , realité : 4
prediction 4  , realité : 3
prediction 3  , realité : 0
prediction 3  , realité : 4
prediction 2  , realité : 2
prediction 3  , realité : 3
prediction 4  , realité : 4


1.0042039071316506